In [17]:
from thinkbayes import Pmf
import thinkbayes as tb

In [18]:
pmf = Pmf()
for x in [1,2,3,4,5,6]:
    pmf.Set(x, 1/6)
    

In [19]:
pmf.GetDict()


{1: 0.16666666666666666,
 2: 0.16666666666666666,
 3: 0.16666666666666666,
 4: 0.16666666666666666,
 5: 0.16666666666666666,
 6: 0.16666666666666666}

In [20]:
tb.MakeHistFromDict(pmf.GetDict())

AttributeError: 'Hist' object has no attribute 'Normalize'